# Baseline Model Training

* Custom Tokenizer
* GRU Encoder / Decoder

## Imports

In [1]:
import os
import io
import sys


import pandas as pd 
import numpy as np 

sys.path.append('D:\PROJECT\Level-4-Project')
os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"

In [2]:
from code2text.models.baseline.model import seq2seqTrain, MaskedLoss
from code2text.helper.model import BatchLogs
from code2text.helper.preprocess import tf_lower_and_split_punct

In [3]:
import ijson

In [3]:
import tensorflow as tf
import tensorflow_text as text

In [4]:
#tf.debugging.set_log_device_placement(True)
tf.config.experimental.set_memory_growth(tf.config.experimental.list_physical_devices('GPU')[0], True)
#tf.config.experimental.set_virtual_device_configuration(tf.config.experimental.list_physical_devices('GPU')[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=3072)])

In [4]:
tf.config.run_functions_eagerly(False)

## Preprocessing

In [4]:
path = "D:\PROJECT\data\CodeSearchNet"
langs = ["java", "python"]
format = ["train.jsonl", "test.jsonl", "valid.jsonl"]

In [5]:
def read_data(path):
    with open(path, encoding="UTF-8") as json_file:
        cursor = 0
        code_set = []
        string_set = []
        lang_set = []
        for _, line in enumerate(json_file):
            code = None
            string = None
            line_as_file = io.StringIO(line)
            json_parser = ijson.parse(line_as_file)

            for prefix, _, value in json_parser:
                if prefix == "code":  
                    code = value
                if prefix == "docstring":
                    string = value
                if prefix == "language":
                    lang = value
            if (code is not None and string is not None):
                code_set.append(code)
                string_set.append(string)
                lang_set.append(lang)

            cursor += len(line)
    return pd.DataFrame(data={'code': code_set, 'docstring': string_set, 'language': lang_set})

In [6]:
train = []
test = []
valid = []

for lang in langs:
    tmp = os.path.join(path, lang)
    for file in format:
        print("Processing ", lang, " ", file, "... ")
        data = read_data(os.path.join(tmp, file))
        if file == "train.jsonl":
            train.append(data)
        if file == "test.jsonl":
            test.append(data)
        if file == "valid.jsonl":
            valid.append(data)

Processing  java   train.jsonl ... 
Processing  java   test.jsonl ... 
Processing  java   valid.jsonl ... 
Processing  python   train.jsonl ... 
Processing  python   test.jsonl ... 
Processing  python   valid.jsonl ... 


In [ ]:
train = pd.concat(train)
test = pd.concat(test)
valid = pd.concat(valid)

In [ ]:
data = pd.concat([train, test, valid])

In [ ]:
train.reset_index(inplace=True)
train.to_json("D:\PROJECT\data\CodeSearchNet\Azure\\train.jsonl", orient="records", lines=True)
test.reset_index(inplace=True)
test.to_json("D:\PROJECT\data\CodeSearchNet\Azure\\test.jsonl", orient="records", lines=True)
valid.reset_index(inplace=True)
valid.to_json("D:\PROJECT\data\CodeSearchNet\Azure\\valid.jsonl", orient="records", lines=True)

In [ ]:
data.reset_index(inplace=True)
data.to_json("D:\PROJECT\data\CodeSearchNet\Combine_clean\data.json")

## Dataset Initialization

In [6]:
data = pd.read_json("D:\PROJECT\data\CodeSearchNet\Combine_clean\data.json")

In [5]:
train = pd.read_json("D:\PROJECT\data\CodeSearchNet\Combine_clean\\train.json")
valid = pd.read_json("D:\PROJECT\data\CodeSearchNet\Combine_clean\\valid.json")
test = pd.read_json("D:\PROJECT\data\CodeSearchNet\Combine_clean\\test.json")

In [6]:
batch_size = 256
buffer = 2048

In [ ]:
dataset = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(data["code"].values, tf.string),
            tf.cast(data["docstring"].values, tf.string)
        )
    )
).shuffle(buffer).batch(batch_size)

In [7]:
train_set = (
    tf.data.Dataset.from_tensor_slices(
        (
            tf.cast(train["code"].values, tf.string),
            tf.cast(train["docstring"].values, tf.string)
        )
    )
).shuffle(buffer).batch(batch_size, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
test_set = (
    tf.data.Dataset.from_tensor_slices(
        (
        tf.cast(test["code"].values, tf.string),
        tf.cast(test["docstring"].values, tf.string)
        )
    )
).shuffle(buffer).batch(batch_size, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)
valid_set = (
    tf.data.Dataset.from_tensor_slices(
        (
        tf.cast(valid["code"].values, tf.string),
        tf.cast(valid["docstring"].values, tf.string)
        )  
    )
).shuffle(buffer).batch(batch_size, drop_remainder=True).cache().prefetch(tf.data.AUTOTUNE)

## Config & Build

### Adapt Process

In [13]:
tokens = 30000
input_processor = input_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=tokens)

output_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    max_tokens=tokens)

In [14]:
input_processor.adapt(train["code"])
output_processor.adapt(train["docstring"])

In [26]:
with open("D:\PROJECT\Level-4-Project\data\invocab.txt", "w") as infile:
    infile.write("\n".join(map(str, input_processor.get_vocabulary())))

with open("D:\PROJECT\Level-4-Project\data\outvocab.txt", "w") as outfile:
    outfile.write("\n".join(map(str, output_processor.get_vocabulary())))

### Build

In [8]:
input_processor = input_text_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    vocabulary="D:\PROJECT\Level-4-Project\data\outvocab.txt")

output_processor = tf.keras.layers.TextVectorization(
    standardize=tf_lower_and_split_punct,
    vocabulary="D:\PROJECT\Level-4-Project\data\outvocab.txt")

In [9]:
train_model = seq2seqTrain(112, 64, input_text_processor=input_processor,
    output_text_processor=output_processor)

In [10]:
train_model.compile(
    optimizer=tf.optimizers.Adam(learning_rate=0.0004),
    loss=MaskedLoss(),
    metrics=['acc', text.metrics.rouge_l]
)

In [11]:
batch_loss = BatchLogs('batch_loss')
chkpt = tf.keras.callbacks.ModelCheckpoint(
    "D:\PROJECT\Level-4-Project\notebooks\training\chkpt\baseline", monitor='loss', save_best_only=True, save_freq=1000
)

## Training

In [14]:
tf.test.is_gpu_available(
    cuda_only=False,
    min_cuda_compute_capability=None
)

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [12]:
tf.keras.backend.clear_session()

In [13]:
history = train_model.fit(train_set, epochs=3, validation_data=valid_set, callbacks=[batch_loss, chkpt])

Epoch 1/3
 2050/14190 [===>..........................] - ETA: 14:46:15 - batch_loss: 5.4566

ResourceExhaustedError: Graph execution error:

Detected at node 'while/decoder/attention/additive_attention/Sum' defined at (most recent call last):
    File "D:\CONDA\envs\code2text\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "D:\CONDA\envs\code2text\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "D:\CONDA\envs\code2text\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "D:\CONDA\envs\code2text\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "D:\CONDA\envs\code2text\lib\site-packages\ipykernel\kernelapp.py", line 677, in start
      self.io_loop.start()
    File "D:\CONDA\envs\code2text\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "D:\CONDA\envs\code2text\lib\asyncio\base_events.py", line 596, in run_forever
      

## Validation & Testing

In [ ]:
history